In [ ]:
# Imports
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import shutil
import csv
import yaml
from scipy import stats
import math
import datetime as dt


In [ ]:
toptoplevel = os.path.abspath('..')
toplevel = os.path.dirname(toptoplevel)
if not toplevel in sys.path:
    sys.path.append(toplevel)

from df_loader import main
COL_NAMES = df_loader.METRICS


In [ ]:
def prepareFolder(path):
    if os.path.exists(path+'/') == True:
        shutil.rmtree(path)
    if os.path.exists(path+'/') == False:
        os.makedirs(path+'/')


In [ ]:
from df_loader import main

dfs, names = main()
# sanity check
for (df, name) in zip(dfs, names):
    print(df['usr'].iloc[0], name)

df = dfs[0]
name = names[0]

# MTD trigger point
mw_tp_rk = df.index.get_indexer(
    df['2022-08-29 08:44:00':'2022-08-29 08:44:00'].index)[0]
mtd_tp_mtd3_start = df.index.get_indexer(
    df['2022-08-29 08:44:07':'2022-08-29 08:44:07'].index)[0]
mtd_tp_mtd3_end = df.index.get_indexer(
    df['2022-08-29 08:44:07':'2022-08-29 08:44:07'].index)[0]

mw_tp_rw = df.index.get_indexer(
    df['2022-08-29 08:45:07':'2022-08-29 08:45:07'].index)[0]
mtd_tp_mtd1_start = df.index.get_indexer(
    df['2022-08-29 08:45:18':'2022-08-29 08:45:18'].index)[0]
mtd_tp_mtd1_end = df.index.get_indexer(
    df['2022-08-29 08:47:42':'2022-08-29 08:47:42'].index)[0]

mw_tp_cc = df.index.get_indexer(
    df['2022-08-29 08:48:23':'2022-08-29 08:48:23'].index)[0]
mtd_tp_mtd2_start = df.index.get_indexer(
    df['2022-08-29 08:48:53':'2022-08-29 08:48:53'].index)[0]
mtd_tp_mtd2_end = df.index.get_indexer(
    df['2022-08-29 08:49:02':'2022-08-29 08:49:02'].index)[0]

# Malware: red
    # trigger: solid
    # download: dotted

# MTD: green
    # MTD 1 : dotted (Ransomware)
    # MTD 2 : solid (CnC)
    # MTD 3 : dashdot (Rootkit) 
    # MTD 4 : solid (CnC)



In [ ]:
# mixed
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend


start = df.index[0]
# print(df.index[0])
if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)
else:
    shutil.rmtree('plots/timeseries/'+name)
    os.mkdir('plots/timeseries/'+name)
index = 0

for column in df.columns:
    # print(column)
    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # BEURK
    df[column+'-avg-healthy'] = df[column].iloc[:mw_tp_rk].mean()
    df[column+'-avg-infected-rk'] = df[column].iloc[mw_tp_rk:mtd_tp_mtd3_start].mean()
    #df[column+'-avg-infected-mtd3-deployment'] = df[column].iloc[mtd_tp_mtd3_start:mtd_tp_mtd3_end].mean()
    df[column+'-avg-cleansed-rk'] = df[column].iloc[mtd_tp_mtd3_end:mw_tp_rw].mean()

    # Ransomware-PoC
    df[column+'-avg-infected-rw'] = df[column].iloc[mw_tp_rw:mtd_tp_mtd1_start].mean()
    df[column+'-avg-infected-mtd1-deployment'] = df[column].iloc[mtd_tp_mtd1_start:mtd_tp_mtd1_end].mean()
    df[column+'-avg-cleansed-rw'] = df[column].iloc[mtd_tp_mtd1_end:mw_tp_cc].mean()

    # httpBackdoor
    df[column+'-avg-infected-cc'] = df[column].iloc[mw_tp_cc:mtd_tp_mtd2_start].mean()
    df[column+'-avg-infected-mtd2-deployment'] = df[column].iloc[mtd_tp_mtd2_start:mtd_tp_mtd2_end].mean()
    df[column+'-avg-cleansed-cc'] = df[column].iloc[mtd_tp_mtd2_end:].mean()

    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = plt.axvline(df.index[mw_tp_rk], color='C2',
                      linewidth=2, linestyle='solid')  # triggering malware
    ax1 = plt.axvspan(
        df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start], alpha=0.3, color='C2')

    ax1 = plt.axvline(df.index[mw_tp_rw], color='C2',
                      linewidth=2, linestyle='dashed')  # triggering malware
    ax1 = plt.axvspan(
        df.index[mw_tp_rw], df.index[mtd_tp_mtd1_start], alpha=0.3, color='C2')

    ax1 = plt.axvline(df.index[mw_tp_cc], color='C2',
                      linewidth=2, linestyle='dotted')  # triggering malware
    ax1 = plt.axvspan(
        df.index[mw_tp_cc], df.index[mtd_tp_mtd2_start], alpha=0.3, color='C2')

    ax1 = plt.axvline(df.index[mtd_tp_mtd3_start], color='C1',
                      linewidth=2, linestyle='dashed')  # triggering MTD 3
    # ax1 = plt.axvline(df.index[mtd_tp_mtd3_end], color='C1', linewidth=2, linestyle='dashed')  # triggering MTD 3

    ax1 = plt.axvline(df.index[mtd_tp_mtd1_start], color='C1',
                      linewidth=2, linestyle='dotted')  # triggering MTD 1
    ax1 = plt.axvspan(df.index[mtd_tp_mtd1_start],
                      df.index[mtd_tp_mtd1_end], alpha=0.3, color='C1')
    ax1 = plt.axvline(df.index[mtd_tp_mtd1_end], color='C1',
                      linewidth=2, linestyle='dotted')  # triggering MTD 1

    ax1 = plt.axvline(df.index[mtd_tp_mtd2_start], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD 1
    ax1 = plt.axvspan(df.index[mtd_tp_mtd2_start],
                      df.index[mtd_tp_mtd2_end], alpha=0.3, color='C1')
    ax1 = plt.axvline(df.index[mtd_tp_mtd2_end], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD 1

    ax1 = df[column].plot(color='C0',
                          alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)

    ax2 = plt.axvspan(df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start],
                      alpha=0.3, color='C2', label='detection phase')
    ax2 = plt.axvspan(df.index[mtd_tp_mtd1_start], df.index[mtd_tp_mtd1_end],
                      alpha=0.3, color='C1', label='deplyoment phase')

    ax2 = plt.axvline(df.index[mw_tp_rk], color='C2', linewidth=2,
                      linestyle='solid', label='start BEURK')  # triggering malware
    #ax2 = plt.axvspan(df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start], alpha=0.3, color='C2')
    ax2 = plt.axvline(df.index[mtd_tp_mtd3_start], color='C1', linewidth=2,
                      linestyle='dashed', label='deploying MTD3')  # triggering MTD 3
    ax2 = plt.axvline(df.index[mtd_tp_mtd3_end], color='C1', linewidth=2,
                      linestyle='dashed', label='finished MTD3')  # triggering MTD 3

    ax2 = plt.axvline(df.index[mw_tp_rw], color='C2', linewidth=2,
                      linestyle='dashed', label='start Ransomware-PoC')  # triggering malware
    ax2 = plt.axvspan(
        df.index[mw_tp_rw], df.index[mtd_tp_mtd1_start], alpha=0.3, color='C2')
    ax2 = plt.axvline(df.index[mtd_tp_mtd1_start], color='C1', linewidth=2,
                      linestyle='dotted', label='deploying MTD1')  # triggering MTD 1
    #ax2 = plt.axvspan(df.index[mtd_tp_mtd1_start], df.index[mtd_tp_mtd1_end], alpha=0.3, color='C1')
    ax2 = plt.axvline(df.index[mtd_tp_mtd1_end], color='C1', linewidth=2,
                      linestyle='dotted', label='finished MTD1')  # triggering MTD 1

    ax2 = plt.axvline(df.index[mw_tp_cc], color='C2', linewidth=2,
                      linestyle='dotted', label='start httpBackdoor')  # triggering malware
    ax2 = plt.axvspan(
        df.index[mw_tp_cc], df.index[mtd_tp_mtd2_start], alpha=0.3, color='C2')
    ax2 = plt.axvline(df.index[mtd_tp_mtd2_start], color='C1', linewidth=2,
                      linestyle='solid', label='deploying MTD2')  # triggering MTD 2
    ax2 = plt.axvspan(df.index[mtd_tp_mtd2_start],
                      df.index[mtd_tp_mtd2_end], alpha=0.3, color='C1')
    ax2 = plt.axvline(df.index[mtd_tp_mtd2_end], color='C1', linewidth=2,
                      linestyle='solid', label='finished MTD2')  # triggering MTD 2

    # BEURK
    ax2 = df[column+'-avg-healthy'].iloc[:mw_tp_rk].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected-rk'].iloc[mw_tp_rk:mtd_tp_mtd3_start].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    # ax2 = df[column+'-avg-infected-mtd3-deployment'].iloc[mtd_tp_mtd3_start:mtd_tp_mtd3_end].plot(
    # color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed-rk'].iloc[mtd_tp_mtd3_end:mw_tp_rw].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    # Ransomware-PoC
    ax2 = df[column+'-avg-infected-rw'].iloc[mw_tp_rw:mtd_tp_mtd1_start].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected-mtd1-deployment'].iloc[mtd_tp_mtd1_start:mtd_tp_mtd1_end].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed-rw'].iloc[mtd_tp_mtd1_end:mw_tp_cc].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    # httpBackdoor
    ax2 = df[column+'-avg-infected-cc'].iloc[mw_tp_cc:mtd_tp_mtd2_start].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-infected-mtd2-deployment'].iloc[mtd_tp_mtd2_start:mtd_tp_mtd2_end].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-avg-cleansed-cc'].iloc[mtd_tp_mtd2_end:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)

    ax3 = plt.axvline(df.index[mw_tp_rk], color='C2',
                      linewidth=2, linestyle='solid')  # triggering malware
    ax3 = plt.axvspan(
        df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start], alpha=0.3, color='C2')
    ax3 = plt.axvline(df.index[mtd_tp_mtd3_start], color='C1',
                      linewidth=2, linestyle='dashed')  # triggering MTD 3
    # ax3 = plt.axvline(df.index[mtd_tp_mtd3_end], color='C1', linewidth=2, linestyle='dashed')  # triggering MTD 3

    ax3 = plt.axvline(df.index[mw_tp_rw], color='C2',
                      linewidth=2, linestyle='dashed')  # triggering malware
    ax3 = plt.axvspan(
        df.index[mw_tp_rw], df.index[mtd_tp_mtd1_start], alpha=0.3, color='C2')
    ax3 = plt.axvline(df.index[mtd_tp_mtd1_start], color='C1',
                      linewidth=2, linestyle='dotted')  # triggering MTD 1
    ax3 = plt.axvspan(df.index[mtd_tp_mtd1_start],
                      df.index[mtd_tp_mtd1_end], alpha=0.3, color='C1')
    ax3 = plt.axvline(df.index[mtd_tp_mtd1_end], color='C1',
                      linewidth=2, linestyle='dotted')  # triggering MTD 1

    ax3 = plt.axvline(df.index[mw_tp_cc], color='C2',
                      linewidth=2, linestyle='dotted')  # triggering malware
    ax3 = plt.axvspan(
        df.index[mw_tp_cc], df.index[mtd_tp_mtd2_start], alpha=0.3, color='C2')
    ax3 = plt.axvline(df.index[mtd_tp_mtd2_start], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD 1
    ax3 = plt.axvspan(df.index[mtd_tp_mtd2_start],
                      df.index[mtd_tp_mtd2_end], alpha=0.3, color='C1')
    ax3 = plt.axvline(df.index[mtd_tp_mtd2_end], color='C1',
                      linewidth=2, linestyle='solid')  # triggering MTD 1
    # BEURK
    ax3 = df[column+'-avg-healthy'].iloc[:mw_tp_rk].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
    ax3 = df[column+'-avg-infected-rk'].iloc[mw_tp_rk:mtd_tp_mtd3_start].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    # ax3 = df[column+'-avg-infected-mtd3-deployment'].iloc[mtd_tp_mtd3_start:mtd_tp_mtd3_end].plot(
    # color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed-rk'].iloc[mtd_tp_mtd3_end:mw_tp_rw].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    # Ransomware-PoC
    ax3 = df[column+'-avg-infected-rw'].iloc[mw_tp_rw:mtd_tp_mtd1_start].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-infected-mtd1-deployment'].iloc[mtd_tp_mtd1_start:mtd_tp_mtd1_end].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed-rw'].iloc[mtd_tp_mtd1_end:mw_tp_cc].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    # httpBackdoor
    ax3 = df[column+'-avg-infected-cc'].iloc[mw_tp_cc:mtd_tp_mtd2_start].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-infected-mtd2-deployment'].iloc[mtd_tp_mtd2_start:mtd_tp_mtd2_end].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax3 = df[column+'-avg-cleansed-cc'].iloc[mtd_tp_mtd2_end:].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    upperBound = df[column].max()
    lowerBound = df[column].min()
    '''
        upperBound = max([df[column+'-avg-healthy'].max(),
                        df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
    lowerBound = min([df[column+'-avg-healthy'].min(),
                        df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])
    '''

    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]

    # shared legend
    '''
    figLegend.legend(lines[1:], labels[1:], bbox_to_anchor=(0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5, ncol=2)
    figLegend = plt.figure(figsize=(5,1.7)) 
    figLegend.savefig('plots/timeseries/{}/{}{}-timeseries-legend.pdf'.format(name, name[3:], index))
    '''

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    # shared legend

    plt.cla()
plt.close('all')


In [ ]:
# legend

column = 'usr'
index = 0
# print(column)
fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

# BEURK
df[column+'-avg-healthy'] = df[column].iloc[:mw_tp_rk].mean()
df[column+'-avg-infected-rk'] = df[column].iloc[mw_tp_rk:mtd_tp_mtd3_start].mean()
#df[column+'-avg-infected-mtd3-deployment'] = df[column].iloc[mtd_tp_mtd3_start:mtd_tp_mtd3_end].mean()
df[column+'-avg-cleansed-rk'] = df[column].iloc[mtd_tp_mtd3_end:mw_tp_rw].mean()

# Ransomware-PoC
df[column+'-avg-infected-rw'] = df[column].iloc[mw_tp_rw:mtd_tp_mtd1_start].mean()
df[column+'-avg-infected-mtd1-deployment'] = df[column].iloc[mtd_tp_mtd1_start:mtd_tp_mtd1_end].mean()
df[column+'-avg-cleansed-rw'] = df[column].iloc[mtd_tp_mtd1_end:mw_tp_cc].mean()

# httpBackdoor
df[column+'-avg-infected-cc'] = df[column].iloc[mw_tp_cc:mtd_tp_mtd2_start].mean()
df[column+'-avg-infected-mtd2-deployment'] = df[column].iloc[mtd_tp_mtd2_start:mtd_tp_mtd2_end].mean()
df[column+'-avg-cleansed-cc'] = df[column].iloc[mtd_tp_mtd2_end:].mean()

df[column+'-rolling-avg'] = df[column].rolling(10).mean()
df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
    10-1::10]
df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

ax1 = plt.subplot(311)
ax1 = plt.axvline(df.index[mw_tp_rk], color='C2',
                  linewidth=2, linestyle='solid')  # triggering malware
ax1 = plt.axvspan(
    df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start], alpha=0.3, color='C2')

ax1 = plt.axvline(df.index[mw_tp_rw], color='C2',
                  linewidth=2, linestyle='dashed')  # triggering malware
ax1 = plt.axvspan(
    df.index[mw_tp_rw], df.index[mtd_tp_mtd1_start], alpha=0.3, color='C2')

ax1 = plt.axvline(df.index[mw_tp_cc], color='C2',
                  linewidth=2, linestyle='dotted')  # triggering malware
ax1 = plt.axvspan(
    df.index[mw_tp_cc], df.index[mtd_tp_mtd2_start], alpha=0.3, color='C2')

ax1 = plt.axvline(df.index[mtd_tp_mtd3_start], color='C1',
                  linewidth=2, linestyle='dashed')  # triggering MTD 3
# ax1 = plt.axvline(df.index[mtd_tp_mtd3_end], color='C1', linewidth=2, linestyle='dashed')  # triggering MTD 3

ax1 = plt.axvline(df.index[mtd_tp_mtd1_start], color='C1',
                  linewidth=2, linestyle='dotted')  # triggering MTD 1
ax1 = plt.axvspan(df.index[mtd_tp_mtd1_start],
                  df.index[mtd_tp_mtd1_end], alpha=0.3, color='C1')
ax1 = plt.axvline(df.index[mtd_tp_mtd1_end], color='C1',
                  linewidth=2, linestyle='dotted')  # triggering MTD 1

ax1 = plt.axvline(df.index[mtd_tp_mtd2_start], color='C1',
                  linewidth=2, linestyle='solid')  # triggering MTD 1
ax1 = plt.axvspan(df.index[mtd_tp_mtd2_start],
                  df.index[mtd_tp_mtd2_end], alpha=0.3, color='C1')
ax1 = plt.axvline(df.index[mtd_tp_mtd2_end], color='C1',
                  linewidth=2, linestyle='solid')  # triggering MTD 1

ax1 = df[column].plot(color='C0',
                      alpha=1, linewidth=1, label=COL_NAMES[index].split(':')[0])

plt.xlabel('')  # override
upperBound = df[column].max()
lowerBound = df[column].min()
if float(lowerBound) == 0 and float(upperBound) == 0:
    ax1 = plt.ylim(bottom=-1, top=1)
else:
    ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                   top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
plt.tick_params(axis='x', which='both', bottom=True,
                top=False, labelbottom=False)
plt.locator_params(axis='y', nbins=6)
plt.locator_params(axis='x', nbins=6)
#plt.title('Metric {}'.format(column))

plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
           borderaxespad=0, fancybox=True, framealpha=0.5)

ax2 = plt.subplot(312)

ax2 = plt.axvspan(df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start],
                  alpha=0.3, color='C2', label='detection phase')
ax2 = plt.axvspan(df.index[mtd_tp_mtd1_start], df.index[mtd_tp_mtd1_end],
                  alpha=0.3, color='C1', label='deplyoment phase')

ax2 = plt.axvline(df.index[mw_tp_rk], color='C2', linewidth=2,
                  linestyle='solid', label='start BEURK')  # triggering malware
#ax2 = plt.axvspan(df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start], alpha=0.3, color='C2')
ax2 = plt.axvline(df.index[mtd_tp_mtd3_start], color='C1', linewidth=2,
                  linestyle='dashed', label='deploying MTD3')  # triggering MTD 3
ax2 = plt.axvline(df.index[mtd_tp_mtd3_end], color='C1', linewidth=2,
                  linestyle='dashed', label='finished MTD3')  # triggering MTD 3

ax2 = plt.axvline(df.index[mw_tp_rw], color='C2', linewidth=2,
                  linestyle='dashed', label='start Ransomware-PoC')  # triggering malware
ax2 = plt.axvspan(
    df.index[mw_tp_rw], df.index[mtd_tp_mtd1_start], alpha=0.3, color='C2')
ax2 = plt.axvline(df.index[mtd_tp_mtd1_start], color='C1', linewidth=2,
                  linestyle='dotted', label='deploying MTD1')  # triggering MTD 1
#ax2 = plt.axvspan(df.index[mtd_tp_mtd1_start], df.index[mtd_tp_mtd1_end], alpha=0.3, color='C1')
ax2 = plt.axvline(df.index[mtd_tp_mtd1_end], color='C1', linewidth=2,
                  linestyle='dotted', label='finished MTD1')  # triggering MTD 1

ax2 = plt.axvline(df.index[mw_tp_cc], color='C2', linewidth=2,
                  linestyle='dotted', label='start httpBackdoor')  # triggering malware
ax2 = plt.axvspan(
    df.index[mw_tp_cc], df.index[mtd_tp_mtd2_start], alpha=0.3, color='C2')
ax2 = plt.axvline(df.index[mtd_tp_mtd2_start], color='C1', linewidth=2,
                  linestyle='solid', label='deploying MTD2')  # triggering MTD 2
ax2 = plt.axvspan(df.index[mtd_tp_mtd2_start],
                  df.index[mtd_tp_mtd2_end], alpha=0.3, color='C1')
ax2 = plt.axvline(df.index[mtd_tp_mtd2_end], color='C1', linewidth=2,
                  linestyle='solid', label='finished MTD2')  # triggering MTD 2

# BEURK
ax2 = df[column+'-avg-healthy'].iloc[:mw_tp_rk].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax2 = df[column+'-avg-infected-rk'].iloc[mw_tp_rk:mtd_tp_mtd3_start].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
# ax2 = df[column+'-avg-infected-mtd3-deployment'].iloc[mtd_tp_mtd3_start:mtd_tp_mtd3_end].plot(
# color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax2 = df[column+'-avg-cleansed-rk'].iloc[mtd_tp_mtd3_end:mw_tp_rw].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

# Ransomware-PoC
ax2 = df[column+'-avg-infected-rw'].iloc[mw_tp_rw:mtd_tp_mtd1_start].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax2 = df[column+'-avg-infected-mtd1-deployment'].iloc[mtd_tp_mtd1_start:mtd_tp_mtd1_end].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax2 = df[column+'-avg-cleansed-rw'].iloc[mtd_tp_mtd1_end:mw_tp_cc].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

# httpBackdoor
ax2 = df[column+'-avg-infected-cc'].iloc[mw_tp_cc:mtd_tp_mtd2_start].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax2 = df[column+'-avg-infected-mtd2-deployment'].iloc[mtd_tp_mtd2_start:mtd_tp_mtd2_end].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax2 = df[column+'-avg-cleansed-cc'].iloc[mtd_tp_mtd2_end:].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
    color='C6', alpha=1, linewidth=1, label='rolling average')

plt.xlabel('')  # override

upperBound = df[column+'-rolling-non-overlapping-avg'].max()
lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
if float(lowerBound) == 0 and float(upperBound) == 0:
    ax2 = plt.ylim(bottom=-1, top=1)
else:
    ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                   top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
plt.tick_params(axis='x', which='both', bottom=True,
                top=False, labelbottom=False)
plt.locator_params(axis='y', nbins=6)
plt.locator_params(axis='x', nbins=6)
#plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

ax3 = plt.subplot(313)

ax3 = plt.axvline(df.index[mw_tp_rk], color='C2',
                  linewidth=2, linestyle='solid')  # triggering malware
ax3 = plt.axvspan(
    df.index[mw_tp_rk], df.index[mtd_tp_mtd3_start], alpha=0.3, color='C2')
ax3 = plt.axvline(df.index[mtd_tp_mtd3_start], color='C1',
                  linewidth=2, linestyle='dashed')  # triggering MTD 3
# ax3 = plt.axvline(df.index[mtd_tp_mtd3_end], color='C1', linewidth=2, linestyle='dashed')  # triggering MTD 3

ax3 = plt.axvline(df.index[mw_tp_rw], color='C2',
                  linewidth=2, linestyle='dashed')  # triggering malware
ax3 = plt.axvspan(
    df.index[mw_tp_rw], df.index[mtd_tp_mtd1_start], alpha=0.3, color='C2')
ax3 = plt.axvline(df.index[mtd_tp_mtd1_start], color='C1',
                  linewidth=2, linestyle='dotted')  # triggering MTD 1
ax3 = plt.axvspan(df.index[mtd_tp_mtd1_start],
                  df.index[mtd_tp_mtd1_end], alpha=0.3, color='C1')
ax3 = plt.axvline(df.index[mtd_tp_mtd1_end], color='C1',
                  linewidth=2, linestyle='dotted')  # triggering MTD 1

ax3 = plt.axvline(df.index[mw_tp_cc], color='C2',
                  linewidth=2, linestyle='dotted')  # triggering malware
ax3 = plt.axvspan(
    df.index[mw_tp_cc], df.index[mtd_tp_mtd2_start], alpha=0.3, color='C2')
ax3 = plt.axvline(df.index[mtd_tp_mtd2_start], color='C1',
                  linewidth=2, linestyle='solid')  # triggering MTD 1
ax3 = plt.axvspan(df.index[mtd_tp_mtd2_start],
                  df.index[mtd_tp_mtd2_end], alpha=0.3, color='C1')
ax3 = plt.axvline(df.index[mtd_tp_mtd2_end], color='C1',
                  linewidth=2, linestyle='solid')  # triggering MTD 1
# BEURK
ax3 = df[column+'-avg-healthy'].iloc[:mw_tp_rk].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average per phase')
ax3 = df[column+'-avg-infected-rk'].iloc[mw_tp_rk:mtd_tp_mtd3_start].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
# ax3 = df[column+'-avg-infected-mtd3-deployment'].iloc[mtd_tp_mtd3_start:mtd_tp_mtd3_end].plot(
# color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax3 = df[column+'-avg-cleansed-rk'].iloc[mtd_tp_mtd3_end:mw_tp_rw].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

# Ransomware-PoC
ax3 = df[column+'-avg-infected-rw'].iloc[mw_tp_rw:mtd_tp_mtd1_start].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax3 = df[column+'-avg-infected-mtd1-deployment'].iloc[mtd_tp_mtd1_start:mtd_tp_mtd1_end].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax3 = df[column+'-avg-cleansed-rw'].iloc[mtd_tp_mtd1_end:mw_tp_cc].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

# httpBackdoor
ax3 = df[column+'-avg-infected-cc'].iloc[mw_tp_cc:mtd_tp_mtd2_start].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax3 = df[column+'-avg-infected-mtd2-deployment'].iloc[mtd_tp_mtd2_start:mtd_tp_mtd2_end].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
ax3 = df[column+'-avg-cleansed-cc'].iloc[mtd_tp_mtd2_end:].plot(
    color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')

ax3.xaxis.set_major_locator(mdates.MinuteLocator())
ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

# adjust y axis
upperBound = df[column].max()
lowerBound = df[column].min()
'''
    upperBound = max([df[column+'-avg-healthy'].max(),
                    df[column+'-avg-infected'].max(), df[column+'-avg-cleansed'].max()])
lowerBound = min([df[column+'-avg-healthy'].min(),
                    df[column+'-avg-infected'].min(), df[column+'-avg-cleansed'].min()])
'''

if float(lowerBound) == 0 and float(upperBound) == 0:
    ax3 = plt.ylim(bottom=-1, top=1)
else:
    ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                   top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

plt.tick_params(axis='x', which='both', bottom=True,
                top=False, labelbottom=True)
plt.locator_params(axis='y', nbins=6)
plt.locator_params(axis='x', nbins=6)
#plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
plt.xlabel('')  # override
#plt.legend(loc='upper left')

fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)
    
# shared legend
figLegend = plt.figure(figsize=(5,1.7)) 

    
   

lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
figLegend.legend(lines[1:], labels[1:], bbox_to_anchor=(0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5, ncol=2)

plt.savefig('plots/timeseries/{}/{}-legend-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
  


index += 1

plt.cla()
plt.close('all')
